In [1]:
# read the data
import xml.etree.ElementTree as ET
import pandas as pd

import spacy
from spacy import displacy
# load a pre-trained english language model
nlp = spacy.load("en_core_web_sm")

import nltk
#nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon

from sklearn.metrics import confusion_matrix

In [2]:
# Parse the XML file
tree = ET.parse('Restaurants.xml')
root = tree.getroot()

# Create empty lists to store data
sentence_ids = []
texts = []
aspect_terms = []
aspect_pos0 = []
aspect_neg0 = []
aspect_nue0 = []

# Extract data from XML and populate the lists
for sentence in root.findall('sentence'):
    sentence_id = sentence.get('id')
    text = sentence.find('text').text

    aspect_pos = []
    aspect_neg = []
    aspect_nue = []
    
    aspect_terms_elem = sentence.find('aspectTerms')
    if aspect_terms_elem is not None:
        for aspect_term in aspect_terms_elem.findall('aspectTerm'):
            term = aspect_term.get('term')
            polarity = aspect_term.get('polarity')
            from_index = int(aspect_term.get('from'))
            to_index = int(aspect_term.get('to'))
            if polarity == 'positive':
                aspect_pos.append((from_index,to_index))
            elif polarity == 'negative':
                aspect_neg.append((from_index,to_index))
            else:
                aspect_nue.append((from_index,to_index))
            #aspect_term_dict[(from_index,to_index)] = (polarity)
    
    sentence_ids.append(sentence_id)
    texts.append(text)
    aspect_pos0.append(aspect_pos)
    aspect_neg0.append(aspect_neg)
    aspect_nue0.append(aspect_nue)

# Create a DataFrame from the extracted data
data = {
    'Sentence ID': sentence_ids,
    'Text': texts,
    'Aspect_pos': aspect_pos0,
    'Aspect_neg': aspect_neg0,
    'Aspect_nue': aspect_nue0,
}
df = pd.DataFrame(data)
#Set the "Sentence ID" column as the index
df.set_index('Sentence ID', inplace=True)

# remove rows that Aspect_pos, Aspect_neg, Aspect_nue are all empty
df = df[(df['Aspect_pos'].str.len() > 0) | (df['Aspect_neg'].str.len() > 0) | (df['Aspect_nue'].str.len() > 0)]
len(df)

2021

In [3]:
# comvert aspect terms into one token if it is more than one token
def convert_aspect_terms(from_index, to_index, text):
    # extract the aspect term
    aspect_term = text[from_index:to_index]
    # if the aspect term is more than one token, convert it into one token
    aspect_term = aspect_term.replace(' ','_')
    text = text[:from_index] + aspect_term + text[to_index:]
    return text

# replce '-' with '_' in all text
for index, row in df.iterrows():
    row['Text'] = row['Text'].replace('-','_')
# # convert aspect terms into one token if it is more than one token
for index, row in df.iterrows():
    index_list = row['Aspect_pos'] + row['Aspect_neg'] + row['Aspect_nue']
    for from_index, to_index in index_list:
        row['Text'] = convert_aspect_terms(from_index, to_index, row['Text'])

In [4]:
df.head()

,Text,Aspect_pos,Aspect_neg,Aspect_nue
Sentence ID,,,,
3121,But the staff was so horrible to us.,[],"[(8, 13)]",[]
2777,"To be completely fair, the only redeeming fact...","[(57, 61)]",[],[]
1634,"The food is uniformly exceptional, with a very...","[(4, 8), (55, 62)]",[],"[(141, 145)]"
2846,"Not only was the food outstanding, but the lit...","[(17, 21), (51, 56)]",[],[]
1458,Our agreed favorite is the orrechiete_with_sau...,"[(27, 62), (76, 83)]",[],"[(152, 157), (113, 117)]"


In [5]:
# count how many Aspect_pos in total
count_pos = 0
count_neg = 0
count_nue = 0
for index, row in df.iterrows():
    count_pos += len(row['Aspect_pos'])
    count_neg += len(row['Aspect_neg'])
    count_nue += len(row['Aspect_nue'])

print('Total number of Aspect_pos: ', count_pos)
print('Total number of Aspect_neg: ', count_neg)
print('Total number of Aspect_nue: ', count_nue)

Total number of Aspect_pos:  2164
Total number of Aspect_neg:  805
Total number of Aspect_nue:  724


In [6]:
# extract the text by the sentence id
def extract_text_by_id(sentence_id):
    text = nlp(df.loc[sentence_id]['Text'])
    text = str(text)
    return text 

In [7]:
def into_parsed_table(text):
    doc = nlp(text)
    # get dependencies
    parsed = pd.DataFrame(columns=["token", "dep", "head", "head_pos", "children"])
    for token in doc:
        row = pd.DataFrame([{
            "token": token.text,
            "dep": token.dep_,
            "head": token.head.text,
            "head_pos": token.head.pos_,
            "children": str([f"{child}" for child in token.children])
        }])
        parsed = pd.concat([parsed, row], axis=0)
    return parsed

def into_parsed_pic(text):
    doc = nlp(text)
    displacy.render(doc, style='dep', jupyter=True)

In [8]:
# after compare with vader_lexicon, opinion_lexicon is better

# import nltk
# #nltk.download('vader_lexicon')
# from nltk.sentiment import SentimentIntensityAnalyzer
# sia = SentimentIntensityAnalyzer()

# def is_neutral(word):
#     sentiment_scores = sia.polarity_scores(word)
#     del sentiment_scores['compound']
#     return max(sentiment_scores, key=lambda key: sentiment_scores[key]) == 'neu'

# def is_positive(word):
#     sentiment_scores = sia.polarity_scores(word)
#     del sentiment_scores['compound']
#     return max(sentiment_scores, key=lambda key: sentiment_scores[key]) == 'pos'

# def is_negative(word):
#     sentiment_scores = sia.polarity_scores(word)
#     del sentiment_scores['compound']
#     return max(sentiment_scores, key=lambda key: sentiment_scores[key]) == 'neg'

# is_negative('yucky')

In [8]:
# build postive and negative word list
positive_words = opinion_lexicon.positive()
positive_words = list(positive_words)
adding_pos_words = ["yummy","favorite","agreed","expert","chilled","unheralded","impecible","magnificant","tasty","flavorful","try"]
positive_words.extend(adding_pos_words)

negative_words = opinion_lexicon.negative()
negative_words = list(negative_words)
adding_neg_words = ["yucky","tasteless","bland","gross","unappetizing","subpar","plain"]
negative_words.extend(adding_neg_words)

nuetral_words = ["average","tipically","ordinary","standard","usual","typical","unremarkable","regular","common","so-so","passable","mediocre","moderate","neutral"]

In [9]:
# has_neg_dep helper function
# if aspect_term's sibling/head/children contains 'neg' dependency >>  return True
def has_neg_dep(text, aspect_term):
    doc = nlp(text)
    has_neg = False
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            for sibling in token.head.children:
                if sibling.dep_ == 'neg':
                    has_neg = True
                # for subchild in sibling.children:
                #     if subchild.dep_ == 'neg':
                #         has_neg = True
            for child in token.children:
                if child.dep_ == 'neg':
                    has_neg = True
            if token.head.dep_ == 'neg':
                has_neg = True
            # if token.head.head.dep_ == 'neg':
            #     has_neg = True

        if has_neg:
            return True  
    return False

## Positive rules

In [10]:
# positive rule1
# aspect_term's children/subchildren or ancestor are in positive words
def positive_rule1(text, aspect_term, has_neg):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            for child in token.children:
                if (child.text.lower()) in positive_words and not has_neg:
                    return True
                for subchild in child.children:
                    if subchild.dep_ in ['amod','advmod','acomp','attr','nsubj'] and (subchild.text.lower()) in positive_words and not has_neg:
                        return True
            if token.head.text.lower() in positive_words:
                return True
    return False

In [11]:
# positive rule2
# aspect_term's siblings with certain dependency are in positive words
def positive_rule2(text, aspect_term, has_neg):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            for child in token.head.children:
                if child.dep_ in ['amod','advmod','acomp','attr','nsubj','conj'] and (child.text.lower())in positive_words and not has_neg:
                    return True
                for subchild in child.children:
                    if subchild.dep_ in ['amod','advmod','acomp','attr','nsubj','conj'] and (subchild.text.lower()) in positive_words and not has_neg:
                        return True
                  
    return False

In [78]:
# positive rule3
# if aspect_term's ancestor is in certain dependency, keep going up until the ancestor is not in that dependency
# and check that ancestor's children
def positive_rule3(text, aspect_term, has_neg):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            node = token.head
            while node.dep_ in ['conj','compound','prep','pobj']:
                node = node.head
            if node.text.lower() in positive_words and not has_neg:
                return True
            for child in node.children:
                if (child.text.lower()) in positive_words and not has_neg:
                    return True
        
    return False

In [14]:
# think about negation case, but "not bad" is not very positive for my understanding
# tried to add, but not very useful

In [79]:
# evaluate the positive rules
def pos_evaluation():
    outcome =[]
    count = 0
    for index, row in df.iterrows():
        all_aspects = row['Aspect_pos'] + row['Aspect_neg'] + row['Aspect_nue']
        for from_index, to_index in all_aspects:  
            if (from_index, to_index) in row['Aspect_pos']:
                ture_label = "true"#"positive"
            else:
                ture_label = "others"

            aspect_term = row['Text'][from_index:to_index]
            has_neg = has_neg_dep(row['Text'], aspect_term)

            pred1 = positive_rule1(row['Text'], aspect_term, has_neg)
            pred2 = positive_rule2(row['Text'], aspect_term, has_neg)
            pred3 = positive_rule3(row['Text'], aspect_term, has_neg)
            

            if pred2 == True or pred3 == True or pred1 == True:
                pred_label = "true"#"positive"
            else:
                pred_label = "others"

            outcome.append((ture_label, pred_label))
                
    print("all done, total pos terms are 2164")
    return outcome

pos_outcome = pos_evaluation()

all done, total pos terms are 2164


In [80]:
# Extract the ground truth and predictions into separate lists
ground_truth = [item[0] for item in pos_outcome]
predictions = [item[1] for item in pos_outcome]

# Create the confusion matrix
confusion_mat = confusion_matrix(ground_truth, predictions)

# Print the confusion matrix
print(confusion_mat)


[[1185  344]
 [ 680 1484]]


In [81]:
# Calculate precision, recall and F1 score

tn, fp, fn, tp = confusion_matrix(ground_truth, predictions).ravel()

recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Positive Recall:", recall)# 在所有的positive里面，有多少能被判断出来
print("Positive Precision:", precision) #在判断为positive的里面，有多少是对的
print("Positive F1 score:", f1_score)


Positive Recall: 0.6857670979667283
Positive Precision: 0.811816192560175
Positive F1 score: 0.7434869739478958


## Negative rules

In [26]:
# negative rule1
# aspect_term's children/subchildren or ancestor are in negative words
def negative_rule1(text, aspect_term):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            for child in token.children:
                if (child.text.lower()) in negative_words:
                    return True
                
                for subchild in child.children:
                    if subchild.dep_ in ['amod','advmod','acomp','attr','nsubj'] and (subchild.text.lower()) in negative_words:
                        return True
                    
         
            if (token.head.text.lower()) in negative_words:
                return True
            
    return False

In [27]:
# negative rule2
# aspect_term's siblings with certain dependency are in negative words
def negative_rule2(text, aspect_term):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            for child in token.head.children:
                if child.dep_ in ['amod','advmod','acomp','attr','nsubj'] and (child.text.lower()) in negative_words:
                    return True
              
                for subchild in child.children:
                    if subchild.dep_ in ['amod','advmod','acomp','attr','nsubj','conj'] and (subchild.text.lower()) in negative_words:
                        return True
                  
    return False

In [28]:
# negative rule3
# if aspect_term's ancestor is in certain dependency, keep going up until the ancestor is not in that dependency
# and check that ancestor's children
def negative_rule3(text, aspect_term):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            node = token.head
            while node.dep_ in ['conj','compound','prep','pobj']:
                node = node.head
            if node.text.lower() in negative_words:
                return True
           
            
            for child in node.children:
                if (child.text.lower()) in negative_words:
                    return True
                
        
    return False

In [87]:
# check the next sentence that doesn't fit the rules
def neg_evaluation():
    outcome =[]
    for index, row in df.iterrows():
        all_aspects = row['Aspect_pos'] + row['Aspect_neg'] + row['Aspect_nue']
        for from_index, to_index in all_aspects:  
            if (from_index, to_index) in row['Aspect_neg']:
                ture_label = "true"#"negative"
            else:
                ture_label = "others"

            aspect_term = row['Text'][from_index:to_index]

            pred1 = negative_rule1(row['Text'], aspect_term)
            pred2 = negative_rule2(row['Text'], aspect_term)
            pred3 = negative_rule3(row['Text'], aspect_term)
            pred4 = has_neg_dep(row['Text'], aspect_term)
            

            if pred2 == True or pred1 == True or pred3 == True or pred4 == True:
                pred_label = "true"#"negative"
            else:
                pred_label = "others"

            outcome.append((ture_label, pred_label))
                
    print("all done, total neg terms are 805")
    return outcome

neg_outcome = neg_evaluation()

all done, total neg terms are 805


In [88]:
# Extract the ground truth and predictions into separate lists
ground_truth = [item[0] for item in neg_outcome]
predictions = [item[1] for item in neg_outcome]

# Create the confusion matrix
confusion_mat = confusion_matrix(ground_truth, predictions)

# Print the confusion matrix
print(confusion_mat)

[[2625  263]
 [ 479  326]]


In [89]:
# Calculate precision, recall and F1 score

tn, fp, fn, tp = confusion_matrix(ground_truth, predictions).ravel()

recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1_score = 2 * (precision * recall) / (precision + recall)


print("Negative Recall:", recall) # 在所有的negative里面，有多少能被判断出来
print("Negative Precision:", precision) # 在判断为negative的里面，有多少是对的
print("Negative F1 score:", f1_score)

Negative Recall: 0.4049689440993789
Negative Precision: 0.5534804753820034
Negative F1 score: 0.46771879483500717


## Neutral rules

In [48]:
# nuetral rule0
# aspect_term is either positive or negative, but not both >> neutral
def neutral_rule0(text, aspect_term, has_neg):
    is_pos = (positive_rule1(text, aspect_term, has_neg) or positive_rule2(text, aspect_term, has_neg) or positive_rule3(text, aspect_term, has_neg))
    is_neg = (negative_rule1(text, aspect_term) or negative_rule2(text, aspect_term) or negative_rule3(text, aspect_term) or has_neg)
    if is_pos and is_neg:
        return True
    if is_pos or is_neg:
        return False
    return True

In [49]:
# negative rule1
# aspect_term's children/subchildren or ancestor are in neutral words
def neutral_rule1(text, aspect_term):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            for child in token.children:
                if (child.text.lower()) in nuetral_words:
                    return True
                
                for subchild in child.children:
                    if subchild.dep_ in ['amod','advmod','acomp','attr','nsubj'] and (subchild.text.lower()) in nuetral_words:
                        return True
         
            if (token.head.text.lower()) in nuetral_words:
                return True
            
    return False

In [50]:
# neutral rule2
# aspect_term's siblings with certain dependency are in neutral words
def neutral_rule2(text, aspect_term):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            for child in token.head.children:
                if child.dep_ in ['amod','advmod','acomp','attr','nsubj'] and (child.text.lower()) in nuetral_words:
                    return True
                for subchild in child.children:
                    if subchild.dep_ in ['amod','advmod','acomp','attr','nsubj','conj'] and (subchild.text.lower()) in nuetral_words:
                        return True
                  
    return False

In [51]:
# neutral rule3
# if aspect_term's ancestor is in certain dependency, keep going up until the ancestor is not in that dependency
# and check that ancestor's children
def neutral_rule3(text, aspect_term):
    doc = nlp(text)
    for token in doc:
        if token.text.lower() == aspect_term.lower():
            node = token.head
            while node.dep_ in ['conj','compound','prep','pobj']:
                node = node.head
            if node.text.lower() in nuetral_words:
                return True

            for child in node.children:
                if (child.text.lower()) in nuetral_words:
                    return True
        
    return False

In [64]:
# evaluate neutral terms
def neu_evaluation():
    outcome =[]
    count = 0
    for index, row in df.iterrows():
        all_aspects = row['Aspect_pos'] + row['Aspect_neg'] + row['Aspect_nue']
        for from_index, to_index in all_aspects:  
            if (from_index, to_index) in row['Aspect_nue']:
                ture_label = "true"#"neutral"
            else:
                ture_label = "others"

            aspect_term = row['Text'][from_index:to_index]
            has_neg = has_neg_dep(row['Text'], aspect_term)

            pred1 = neutral_rule1(row['Text'], aspect_term)
            pred2 = neutral_rule2(row['Text'], aspect_term)
            pred3 = neutral_rule3(row['Text'], aspect_term)
            pred0 = neutral_rule0(row['Text'], aspect_term, has_neg)


            if pred2 == True or pred1 == True or pred3 == True or pred0 == True: 
                pred_label = "true"#"neutral"
            else:
                pred_label = "others"

            outcome.append((ture_label, pred_label))
                
    print("all done, total pos terms are 724")
    return outcome

neu_outcome = neu_evaluation()

all done, total pos terms are 724


In [65]:
# Extract the ground truth and predictions into separate lists
ground_truth = [item[0] for item in neu_outcome]
predictions = [item[1] for item in neu_outcome]


# Create the confusion matrix
confusion_mat = confusion_matrix(ground_truth, predictions)

print(confusion_mat)


[[1861 1108]
 [ 224  500]]


In [66]:

tn, fp, fn, tp = confusion_matrix(ground_truth, predictions).ravel()

recall = tp / (tp + fn)

precision = tp / (tp + fp)

# Calculate F1 score
f1_score = 2 * (precision * recall) / (precision + recall)


print("Nuetral Recall:", recall)# 在所有的nuetral里面，有多少能被判断出来 
print("Nuetral Precision:", precision)#在判断为neutral的里面，有多少是对的
print("Nuetral F1 score:", f1_score)

Nuetral Recall: 0.6906077348066298
Nuetral Precision: 0.31094527363184077
Nuetral F1 score: 0.42881646655231553
